In [7]:
import pandas as pd
import numpy as np
###### creamos el dataframe ##

#pklName = 'celda1_ciclo 38_23-10-2024.pkl'
pklName = 'celda1_ciclo 2_09-09-2024.pkl'
df1 = pd.read_pickle(pklName)
df1 = df1.rename(columns={"Iup": "iUp", "Idw": "iDw"})       

Ama = 1.0
df1[['iUp', 'iDw']] = df1[['iUp', 'iDw']]*1000 


###### Encontrar Voc #######
## Voc_dw
# Encontrar el índice del valor más cercano a 0 entre los valores positivos de la corriente
positivos_idw = df1[df1['iDw'] > 0].reset_index(drop=True)
row_idw = (positivos_idw['iDw']).idxmin()

voc_dw = df1['Vdw'].iloc[row_idw]

## Voc_up
# Encontrar el índice del valor más cercano a 0 entre los valores negativos de la corriente
negativos_iup = df1[df1['iUp'] < 0].reset_index(drop=True)
row_iup = (negativos_iup['iUp']).idxmax()

voc_up = df1['Vup'].iloc[row_iup]

###### Encontrar jSC ####

## jsc_dw
jsc_dw = df1['iDw'].iloc[48]

## jsc_up
jsc_up = df1['iUp'].iloc[0]

############ CALCULAMOS valores maximos ####

def productMask(df, x_var, y_var):
    """Encuentra la posición (entera) donde el producto absoluto entre dos columnas es máximo."""
    # Filtrar las filas donde y_var < 0
    mask = df[y_var] < 0

    # Calcular el producto absoluto de las columnas filtradas
    product = (df.loc[mask, x_var] * df.loc[mask, y_var]).abs()

    # Obtener el índice del máximo valor
    max_idx = product.idxmax()

    # Convertir el índice al número de posición (entero) para usarlo con iloc
    return df.index.get_loc(max_idx)
    
Vmax_up = df1['Vup'].iloc[productMask(df1, 'Vup', 'iUp')]
jMax_up = df1['iUp'].iloc[productMask(df1, 'Vup', 'iUp')] / Ama

Vmax_dw = df1['Vdw'].iloc[productMask(df1, 'Vdw', 'iDw')]
jMax_dw = df1['iDw'].iloc[productMask(df1, 'Vdw', 'iDw')] / Ama


#### guardamso los valores de voltaje y corriente caracteristicos del ciclo ###

Voc = [voc_up, voc_dw] 
jSC = [jsc_up, jsc_dw]

Vmax = [Vmax_up, Vmax_dw]
jMax = [jMax_up, jMax_dw]

###### calculamos los valores de las funciones FF PCE

def PCE(Vmax, jmax, Pin=100.0):                       # Eficiencia
    return np.abs((100.0*Vmax*jmax) / (Pin))              # [%]  ;  Pin: dato piranómetro ; (lab) P_In = 100 mW/cm^2 = 100 V*mA/cm^2

def FF(Vmax, jmax, Voc, jSC):
    return 100*(Vmax*jmax) / (Voc*jSC)                      # Adimensional

#Calling our functions
FF_up = round(FF(Vmax[0], jMax[0], Voc[0], jSC[0]), 4)
FF_dw = round(FF(Vmax[1], jMax[1], Voc[1], jSC[1]), 4)

PCE_up = round(PCE(Vmax[0], jMax[0]),4)
PCE_dw = round(PCE(Vmax[1], jMax[1]),4)


Ff = [FF_up, FF_dw] ; Pce = [PCE_up, PCE_dw]


####### creamos el dataframe de salida
data = {
    'Voc': pd.Series(Voc), 
    'jSC': pd.Series(jSC), 
    'Vmax': pd.Series(Vmax), 
    'jMax': pd.Series(jMax),
    'FF': pd.Series(Ff),
    'PCE %' : pd.Series(Pce)}
   

df_final = pd.DataFrame(data)
# Asignar los nombres al índice
df_final.index = ['up', 'dw']

print(df_final)


       Voc    jSC    Vmax   jMax       FF   PCE %
up  1.0960 -1.598  0.6535 -1.184  44.1784  0.7737
dw  1.2065 -1.243  0.6535 -0.811  35.3401  0.5300


In [16]:

Vmax = [Vmax_up, Vmax_dw]
jMax = [jMax_up, jMax_dw]

def FF(Vmax, jmax, Voc, jSC):
    return 100*(Vmax*jmax) / (Voc*jSC)                      # Adimensional

FF_up = round(FF(Vmax[0], jMax[0], Voc[0], jSC[0]), 6)
#FF_dw = round(FF(Vmax[1], jMax[1], Voc[1], jSC[1]), 2)
FFUP= 100*(Vmax_up * jMax_up) / (voc_up * jsc_up)
#Ffup = 100 * ( Vmax[0]* jMax[0]) / (Voc[0] * jsc )
print(Vmax[0])
print(Voc[0])
print(FF_up)
print(FFUP)

0.6535
1.096
44.466782
44.46678213534578


In [ ]:
ciclo 38

       Voc    jSC    Vmax   jMax       FF   PCE %     Hi
up  1.1515 -1.947  0.6535 -1.452  42.3236  0.9489  2.293
dw  1.1515 -1.654  0.7090 -1.123  41.8048  0.7962    NaN


In [6]:
#'1.151', '-1.71', '1.151', '-1.98']

    Voc    jSC    Vmax   jMax      FF   PCE %      Hi
up  1.1515 -1.987  0.6535 -1.483  42.357  0.9691  2.2632
dw  1.1515 -1.735  0.7090 -1.174  41.663  0.8324     NaN



Cruces:
JSC   Voc    dw:  -1.7307 , 1.1515
             up : -1.9863 ,  1.1515



ciclo 1  

       Voc    jSC    Vmax   jMax       FF   PCE %
up  1.1515 -1.412  0.6535 -1.030  41.3985  0.6731
dw  1.2065 -1.150  0.6535 -0.707  33.2997  0.4620

IndentationError: unexpected indent (879547576.py, line 3)